In [1]:
import requests
import psycopg2
from psycopg2.extras import execute_values
import uuid

In [4]:
url = 'https://randomuser.me/api/?results=5000'

In [5]:
response=requests.get(url).json()

In [6]:
response = response['results']

In [7]:
response[5]

{'gender': 'male',
 'name': {'title': 'Mr', 'first': 'بردیا', 'last': 'زارعی'},
 'location': {'street': {'number': 277, 'name': 'شهید ثانی'},
  'city': 'اهواز',
  'state': 'مرکزی',
  'country': 'Iran',
  'postcode': 64105,
  'coordinates': {'latitude': '22.0361', 'longitude': '-40.0497'},
  'timezone': {'offset': '+4:00',
   'description': 'Abu Dhabi, Muscat, Baku, Tbilisi'}},
 'email': 'brdy.zraay@example.com',
 'login': {'uuid': 'ad0ae47f-f8fd-469e-b79b-77230307fe13',
  'username': 'bigmeercat564',
  'password': 'campbell',
  'salt': '7LLOx7Gv',
  'md5': 'c791da36b2d571c175b9d46af3952c7b',
  'sha1': '03b18ced360246d8901c7503b3e421199937411f',
  'sha256': 'f41cb31fefdaaafd75ab3467db1730aae56cc799ac69162f7cb72ccce1cbb9f2'},
 'dob': {'date': '1964-02-28T22:05:18.819Z', 'age': 60},
 'registered': {'date': '2015-06-27T13:24:34.689Z', 'age': 9},
 'phone': '013-60268990',
 'cell': '0980-454-3068',
 'id': {'name': '', 'value': None},
 'picture': {'large': 'https://randomuser.me/api/portraits

In [17]:
dataset = []
for res in response:
    data = {}
    location = res['location']
    
    data['id'] = uuid.uuid4()
    data['first_name'] = res['name']['first']
    data['last_name'] = res['name']['last']
    data['gender'] = res['gender']
    data['street_address'] = f"{str(location['street']['number'])} {location['street']['name']}"
    data['city'] = location['city']
    data['state'] = location['state']
    data['country'] = location['country']
    data['postal_code'] = location['postcode']
    data['email'] = res['email']
    data['username'] = res['login']['username']
    data['dob'] = res['dob']['date']
    data['registered_date'] = res['registered']['date']
    data['phone'] = res['phone']
    data['picture'] = res['picture']['medium']
    
    dataset.append(data)
    
ids = [str(data["id"]) for data in dataset]
    
dataset = [
        (
            str(user['id']),
            user['first_name'],
            user['last_name'],
            user['gender'],
            user['street_address'],
            user['city'],
            user['state'],
            user['country'],
            str(user['postal_code']),
            user['email'],
            user['username'],
            user['dob'],
            user['registered_date'],
            user['phone'],
            user['picture']
        )
        for user in dataset
    ]

In [19]:
len(ids)

5000

In [11]:
db_config = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "postgres",
    "host": "localhost",  # or the host of your PostgreSQL server
    "port": 5432  # default PostgreSQL port
}


# Connect to the database
connection = psycopg2.connect(**db_config)
cursor = connection.cursor()



In [13]:
##### Create the users table
connection.rollback()

create_table_query = """

CREATE TABLE IF NOT EXISTS users (
    id UUID PRIMARY KEY,
    first_name VARCHAR(255),
    last_name VARCHAR(255),
    gender VARCHAR(255),
    street_address VARCHAR(255),
    city VARCHAR(255),
    state VARCHAR(255),
    country VARCHAR(255),
    postal_code VARCHAR(255),
    email VARCHAR(255),
    username VARCHAR(255),
    date_of_birth TIMESTAMPTZ,
    registered_date TIMESTAMPTZ,
    phone VARCHAR(255),
    picture VARCHAR(255)

);
"""
cursor.execute(create_table_query)
connection.commit()
print("Users table created successfully.")
connection.commit()


Users table created successfully.


In [14]:
# Insert a few entries
insert_query = """
    INSERT INTO users (
        id, first_name, last_name, gender, street_address,city,state,country, postal_code, email, username, 
        date_of_birth, registered_date, phone, picture
    ) VALUES %s
    """


execute_values(cursor, insert_query, dataset)
connection.commit()